In [1]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir
from k12libs.utils.nb_widget import k12ai_schema_parse

import os
import json
import _jsonnet

In [2]:
cv_schema_dir = os.path.join(k12ai_get_app_dir('cv'), 'templates', 'schema')
cv_json = _jsonnet.evaluate_file(os.path.join(cv_schema_dir, 'k12cv.jsonnet'))
# k12ai_print(cv_json)

In [3]:
k12ai_schema_parse(json.loads(cv_json), debug=False)

Box(children=(Tab(children=(VBox(children=(Accordion(children=(VBox(layout=Layout(align_items='stretch', justi…

Output()